In [1]:
import json
from ckiptagger import construct_dictionary, WS, POS
ws = WS("./data")

C:\Users\Tibame_T14\anaconda3\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\Users\Tibame_T14\anaconda3\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
C:\Users\Tibame_T14\anaconda3\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:993: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(


In [2]:
with open('./youtube/醫療保健text.json') as f:
    data = json.loads(f.read())
with open('./stopword.txt', encoding='utf-8') as f:
    stop = f.read()

In [3]:
stopword = stop.split('\n')

In [4]:
# 使用分好詞的資料
alltext = []
for file in data.values():
    for name, texts in file.items():
        word_sentence_list = ws(list(texts))
        text_list = []
        for i in range(len(texts)):
            for word in word_sentence_list[i]:
                if word not in stopword:
                    text_list.append(word)
        if text_list != []:
            alltext.append(text_list)

In [5]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [6]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(alltext)]
model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4)

In [7]:
from gensim.similarities import WordEmbeddingSimilarityIndex

In [8]:
termsim_index = WordEmbeddingSimilarityIndex(model.wv)

In [9]:
from gensim.corpora import Dictionary

In [10]:
dictionary = Dictionary(alltext)

In [11]:
bow_corpus = [dictionary.doc2bow(document) for document in alltext]

In [12]:
from gensim.similarities import SoftCosineSimilarity, SparseTermSimilarityMatrix

In [13]:
similarity_matrix = SparseTermSimilarityMatrix(termsim_index, dictionary)
docsim_index = SoftCosineSimilarity(bow_corpus, similarity_matrix)

100%|███████████████████████████████████████████████████████████████████████████| 26687/26687 [01:23<00:00, 318.03it/s]


In [14]:
sims = docsim_index[dictionary.doc2bow(['瘦身','健康','減脂'])]
sims

C:\Users\Tibame_T14\anaconda3\lib\site-packages\gensim\similarities\termsim.py:382: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
C:\Users\Tibame_T14\anaconda3\lib\site-packages\gensim\similarities\termsim.py:382: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.7316522 , 1.        , 1.        , 0.9111905 , 1.        ,
       1.        , 0.8301885 , 1.        , 1.        , 1.        ,
       0.99931395, 1.        , 1.        , 0.9013168 , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.69563645, 0.9459089 ,
       0.84872055, 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.79348576, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.8300884 , 1.        , 0.70466214, 0.4967255 , 1.        ,
       0.        , 1.        , 0.        , 1.        , 0.69764215,
       1.        , 0.44240758, 0.91529584, 0.6064992 , 0.87224

In [15]:
model.wv.most_similar(['瘦身','健康','減脂'])

[('益生菌', 0.9936637878417969),
 ('堅持', 0.9919912219047546),
 ('控制', 0.991941511631012),
 ('乖乖', 0.9918344616889954),
 ('吃到', 0.9917516708374023),
 ('嚴格', 0.9913499355316162),
 ('發胖', 0.9911790490150452),
 ('成功', 0.9910815954208374),
 ('試', 0.9904205799102783),
 ('痛苦', 0.9899192452430725)]

In [16]:
model.wv.similar_by_word('瘦身')

[('辦', 0.9978082180023193),
 ('算', 0.9977234601974487),
 ('重點', 0.9975895881652832),
 ('算是', 0.9973477721214294),
 ('裝', 0.9973286986351013),
 ('種子', 0.9972751140594482),
 ('原則', 0.9971699118614197),
 ('設', 0.997025191783905),
 ('免費', 0.9970196485519409),
 ('豬', 0.9969728589057922)]

In [24]:
model.wv[alltext[0]]

array([[-0.02427432, -0.1457555 , -0.5095117 , ..., -0.14277664,
         0.23143183, -0.1292079 ],
       [-0.11492795, -0.20340832, -0.29187506, ..., -0.17336613,
         0.66452545, -0.05533678],
       [-0.2345905 , -0.12320617, -0.34969282, ..., -0.2621024 ,
         0.7981411 ,  0.37003243],
       ...,
       [ 0.08088113, -0.04460506, -0.67256045, ..., -0.15957996,
         0.05120504, -0.22449484],
       [-0.00507432,  0.00852131, -0.01548689, ..., -0.00631102,
        -0.00789565, -0.00368631],
       [-0.01793433, -0.03791821, -0.36582193, ..., -0.13407761,
         0.13370182, -0.0730036 ]], dtype=float32)